In [1]:
!nvidia-smi

Wed Aug 27 12:30:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.6/413.6 kB 21.1 MB/s eta 0:00:00


In [3]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.0
    Uninstalling accelerate-1.10.0:
      Successfully uninstalled accelerate-1.10.0
Found existing installation: transformers 4.55.4
Uninstalling transformers-4.55.4:
  Successfully uninstalled transformers-4.55.4
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
  Using cached transformers-4.55.4-py3-none-any.whl.metadata (41 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.55.4-py3-none-any.whl (11.3 MB)
Using cached accelerate-1.10.1-py3-none-any.whl (374 kB)


In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [6]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


4.55.4
/usr/local/lib/python3.12/dist-packages/transformers/__init__.py


In [7]:
import evaluate

In [8]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import evaluate
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [11]:
model_ckpt = "facebook/bart-base"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegusus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [14]:
from datasets import load_dataset

# Load CNN/DailyMail dataset (v3.0.0 is the latest processed version)
dataset = load_dataset("knkarthick/samsum", trust_remote_code=True)


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'knkarthick/samsum' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'knkarthick/samsum' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [15]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})


In [16]:
# Check dataset sizes
split_lengths = [len(dataset[split]) for split in dataset]
print(f"Split lengths: {split_lengths}")

# Show available columns
print(f"Features: {dataset['train'].column_names}")

# Inspect one sample
print("\nDialogue:")
print(dataset["test"][1]["dialogue"])   # full conversation

print("\nSummary:")
print(dataset["test"][1]["summary"])    # human-written summary


Split lengths: [14732, 818, 819]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [20]:
def convert_examples_to_features(batch):
    # Ensure inputs are strings
    dialogues = [str(x) for x in batch["dialogue"]]
    summaries = [str(x) for x in batch["summary"]]

    # Tokenize the dialogue (inputs)
    input_encodings = tokenizer(
        dialogues,
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

    # Tokenize the summaries (targets)
    target_encodings = tokenizer(
        text_target=summaries,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }

# Apply mapping
dataset_samsum_pt = dataset.map(
    convert_examples_to_features,
    batched=True,
    remove_columns=dataset["train"].column_names  # drop old cols like 'dialogue','summary'
)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [22]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [23]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model_pegusus
)


In [24]:
import transformers
print(transformers.__version__)

import transformers
print(transformers.__version__)
print(transformers.__file__)



4.55.4
4.55.4
/usr/local/lib/python3.12/dist-packages/transformers/__init__.py


In [26]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./checkpoints",        # where to save checkpoints
    eval_strategy="epoch",       # evaluate at end of each epoch
    save_strategy="epoch",             # save every epoch
    save_total_limit=2,                # keep only last 2 checkpoints
    load_best_model_at_end=True,       # restore best model

    # Performance optimizations
    per_device_train_batch_size=8,     # SAMSum is small, GPU should handle 8
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,     # effective batch size = 16
    fp16=True,                         # mixed precision (saves VRAM)
    dataloader_num_workers=2,

    # Training dynamics
    num_train_epochs=5,                # more epochs for smaller dataset
    learning_rate=3e-5,                # smaller models need smaller LR
    weight_decay=0.01,
    warmup_steps=100,                  # fewer steps, dataset is small

    # Logging
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=20,                  # log more frequently (small dataset)
    report_to="none",
    predict_with_generate=True,        # enables .generate() in evaluation
)


# New Section

In [27]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model_pegusus,   # use your T5-small or Pegasus-small model
    args=training_args,
    train_dataset=dataset_samsum_pt["train"],
    eval_dataset=dataset_samsum_pt["validation"],
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
)

/tmp/ipython-input-2077887249.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [28]:
# Train the BART summarizer
train_output = trainer.train()

Epoch,Training Loss,Validation Loss
1,0.377800,0.337639
2,0.319000,0.326614
3,0.305400,0.322634
4,0.295800,0.321993
5,0.270600,0.321846


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


In [29]:
from tqdm import tqdm
import evaluate

# Load ROUGE metric
rouge_metric = evaluate.load("rouge")

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """Split a list into smaller batches of size batch_size."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model_pegusus, tokenizer,
                               batch_size=16, device="cuda",
                               column_text="dialogue",
                               column_summary="summary"):
    # Split dataset into batches of inputs and references
    text_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for text_batch, target_batch in tqdm(
        zip(text_batches, target_batches), total=len(text_batches)
    ):
        # Tokenize inputs
        inputs = tokenizer(
            text_batch, max_length=1024, truncation=True,
            padding="max_length", return_tensors="pt"
        )

        # Generate summaries
        summaries = model_pegusus.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8, num_beams=4, max_length=128
        )

        # Decode summaries into text
        decoded_summaries = [
            tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for s in summaries
        ]

        # Add to metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute final ROUGE score
    score = metric.compute()
    return score

# Run evaluation on SAMSum test set
scores = calculate_metric_on_test_ds(
    dataset_samsum_pt["test"], rouge_metric, model_pegusus, tokenizer, batch_size=8, device="cuda"
)

print(scores)

100%|██████████| 103/103 [02:20<00:00,  1.37s/it]


{'rouge1': np.float64(0.49013423405703793), 'rouge2': np.float64(0.24810404351375695), 'rougeL': np.float64(0.40909490666297993), 'rougeLsum': np.float64(0.40905061184018665)}


In [31]:
import evaluate

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = evaluate.load("rouge")


In [36]:
import pandas as pd

# Define which ROUGE metrics to report
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

# Extract scores (already float values in evaluate)
rouge_dict = {rn: scores[rn] for rn in rouge_names}

# Convert to DataFrame for nicer display
df_scores = pd.DataFrame(rouge_dict, index=["pegasus"])
print(df_scores)


           rouge1    rouge2    rougeL  rougeLsum
pegasus  0.490134  0.248104  0.409095   0.409051


In [38]:
## Save model
model_pegusus.save_pretrained("pegasus-samsum-model")

In [39]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [40]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [42]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum_pt["test"][0]["dialogue"]

reference = dataset_samsum_pt["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Betty called Larry last time they were at the park together.
